This notebook is used to add synthetic data to datasets.\
The current implemented method is adding random ratings to a target user.\

We can take target users from a certain cluster based on the result of the previous notebook (clustering.ipynb) and use this code to generate random ratings to few users inside a certain cluster. This simulates an opt-out scenario.

In [1]:
import os
import pandas as pd

# change dir for custom imports
os.chdir('../')

dataset_name = 'ml-25m'

ratings = pd.read_csv('datasets/' + dataset_name + '/shrunk/ratings_small_v2.csv')
total_movies = len(list(set(ratings.movieId.to_list())))
max_ratings = max(list(set(ratings.rating.to_list())))
min_ratings = min(list(set(ratings.rating.to_list())))

In [ ]:
import numpy as np

# add random ratings to first user in cluster 0
target_user = 2631
N = 350

# user_ratings_df = ratings[ratings['userId'] == target_user]

for n in range(N):
    # new record to add
    new_record = {
        'userId': target_user,
        'movieId': np.random.choice(np.arange(1, total_movies, 1), size=1)[0],
        'rating': np.random.choice(np.arange(1, 5, 0.5), size=1)[0],
        'timestamp': 1212603770
    }
    # append row to the dataframe
    ratings = ratings.append(new_record, ignore_index=True)

In [ ]:
len(ratings)

In [ ]:
ratings.to_csv('datasets/' + dataset_name + '/modified/ratings_random_experiment.csv', index=False)